In [192]:
import numpy as np
import numpy.linalg as LA
from math import *
N = 0
accuracy = [1e-3, 1e-5, 1e-9]
koeffs = [1, 100, 250]
startPoint = [np.array([0, 0]), np.array([-5, 0])]
A = np.array([[-42, 0], [0, -26]], dtype=np.int64)


In [181]:
import sympy
x = sympy.Symbol('x')
y = sympy.Symbol('y')
eq = (x**2+y-11)**2 + (x + y**2 - 7)**2

In [193]:
import pandas as pd
results = pd.DataFrame(columns=['title', 'SP', *[f'EPS={i}' for i in accuracy]])

In [194]:
results

,title,SP,EPS=0.001,EPS=1e-05,EPS=1e-09


In [209]:
def func(x, a):
    global N
    N += 1
    return (x[0] * x[0] + x[1] - 11) ** 2 + (x[0] + x[1] * x[1] - 7) ** 2


def fD0(x, a):
    global N
    N += 1
    return -2*(2*x[0]*(x[0]**2+x[1]-11)+x[0]+x[1]**2-7)


def fD1(x, a):
    global N
    N += 1
    return -2*(x[0]**2+2*x[1]*(x[0]+x[1]**2-7)+x[1]-11)


def gradient(x, a):
    return np.array([fD0(x, a), fD1(x, a)])

# def gradient(arg, a):
#     global N
#     N += 2
#     return np.array([int(sympy.diff(eq,x).subs([(x,arg[0]), (y, arg[1])])), int(sympy.diff(eq,y).subs([(x,arg[0]), (y, arg[1])]))])


In [206]:
def gradientDescent(startPoint, epsilon, koeff, A):

    point = np.array(startPoint)
    grad = gradient(point, koeff)
    

    while (LA.norm(grad) >= epsilon):
        print(point)
        alpha_k = - np.dot(grad, -grad)/np.dot(A @ -grad, -grad)
        point = point - alpha_k * grad/2

        grad = gradient(point, koeff)

    return (point, func(point, koeff))



In [ ]:
for sp in startPoint:
    N_EPS = []
    for eps in accuracy:
        gradientDescent(sp, eps, 1, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"gradientDescent" , 'SP': sp, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1],"EPS=1e-09":N_EPS[2]}).to_frame().T], ignore_index=True)


In [210]:
print(gradientDescent(np.array([0, 0]), 1, 1e-3, A))
print(gradientDescent(np.array([-5, 0]), 1, 1e-3, A))

[0 0]
[0.22867025 0.35933897]
[0.59054749 0.84338746]
[1.11311834 1.43058178]
[1.74960884 1.97985258]
[2.32225285 2.26969747]
[2.6808854  2.29802083]
[2.86107605 2.22490245]
[2.93941573 2.1325039 ]
[2.96892423 2.06824936]
[2.98433125 2.03731212]
(array([2.99146971, 2.01987116]), 0.006066509179158906)
[-5  0]
[-1.36931758 -0.33440496]
[-1.7972417  -0.23895084]
[-2.28106249 -0.14647232]
[-2.7113579  -0.06861113]
[-2.96496034 -0.01072933]
[-3.05028096  0.04278644]
[-3.0516568  0.1394234]
[-3.02941969  0.30474698]
[-3.00112196  0.59437458]
[-2.95326909  1.08645135]
[-2.88688265  1.86199485]
[-2.81870461  2.80942963]
[-2.79789032  3.23443185]
[-2.80989997  3.06743205]
[-2.80251693  3.16306961]
[-2.80658679  3.11334377]
(array([-2.80432389,  3.14082836]), 0.0036833078904603706)


In [199]:
results

,title,SP,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,"[0, 0]",47,63,93
1,gradientDescent,"[-5, 0]",59,75,105


In [200]:
def conjugateGradientMethod(startPoint, epsilon, koeff, A):
    point = np.array(startPoint, dtype=np.int64)
    grad = gradient(point, koeff)
    p_k = -grad
    k = 0

    while (LA.norm(grad) >= epsilon):
        alpha_k = - np.dot(grad, p_k)/np.dot(A @ p_k, p_k)
        point = point + alpha_k * p_k
        Ngrad = gradient(point, koeff)

        if (k + 1 == point.size):
            betta_k=0
        else:
            betta_k = LA.norm(Ngrad)**2/LA.norm(grad)**2
        p_k = -Ngrad + np.dot(betta_k, p_k)

        grad = Ngrad

    return (point, func(point, koeff))


In [201]:
print(conjugateGradientMethod(np.array([0, 0]), 1, 1e-6, A))
print(conjugateGradientMethod(np.array([-5, 0]), 1, 1e-6, A))

(array([3.01389232, 1.99016985]), 0.006075852917306318)
(array([ 3.5886631 , -1.84772621]), 0.0009547935930748398)


In [202]:
for sp in startPoint:
    N_EPS = []
    for eps in accuracy:
        conjugateGradientMethod(sp, eps, 1, A)
        N_EPS.append(N)
        N = 0
    results = pd.concat([results, pd.Series({"title":"conjugateGradientMethod", "SP": sp, "EPS=0.001":N_EPS[0],"EPS=1e-05":N_EPS[1],"EPS=1e-09":N_EPS[2]}).to_frame().T], ignore_index=True)

C:\Users\Bambloo\AppData\Local\Temp/ipykernel_139912/945911776.py:8: RuntimeWarning: invalid value encountered in double_scalars
  alpha_k = - np.dot(grad, p_k)/np.dot(A @ p_k, p_k)


In [203]:
results

,title,SP,EPS=0.001,EPS=1e-05,EPS=1e-09
0,gradientDescent,"[0, 0]",47,63,93
1,gradientDescent,"[-5, 0]",59,75,105
2,conjugateGradientMethod,"[0, 0]",281,161,207
3,conjugateGradientMethod,"[-5, 0]",1459,1459,1459
